In [15]:
liquidity = {
    ("tokenA", "tokenB"): (17, 10),
    ("tokenA", "tokenC"): (11, 7),
    ("tokenA", "tokenD"): (15, 9),
    ("tokenA", "tokenE"): (21, 5),
    ("tokenB", "tokenC"): (36, 4),
    ("tokenB", "tokenD"): (13, 6),
    ("tokenB", "tokenE"): (25, 3),
    ("tokenC", "tokenD"): (30, 12),
    ("tokenC", "tokenE"): (10, 8),
    ("tokenD", "tokenE"): (60, 25),
}
newLq = dict()
for x, y in liquidity.items():
    newLq[x] = y
    newLq[(x[1], x[0])] = (y[1], y[0])
    

def getAmountOut(amountIn, reserveIn, reserveOut):
    amountInWithFee = amountIn * 997
    numerator = amountInWithFee * reserveOut
    denominator = reserveIn * 1000 + amountInWithFee
    amountOut = numerator / denominator
    return amountOut

def swap(tokenIn, tokenOut, amountIn, _liquidity):
    amountOut = getAmountOut(amountIn, _liquidity[(tokenIn, tokenOut)][0], _liquidity[(tokenIn, tokenOut)][1])
    _liquidity[(tokenIn, tokenOut)] = (_liquidity[(tokenIn, tokenOut)][0] + amountIn, _liquidity[(tokenIn, tokenOut)][1] - amountOut)
    _liquidity[(tokenOut, tokenIn)] = (_liquidity[(tokenIn, tokenOut)][1] - amountOut, _liquidity[(tokenIn, tokenOut)][0] + amountIn)
    
def arbitrage(token, amountIn, path, visited, liq):
    if token == "tokenB" and len(path) > 1:
        return amountIn, path
    l = liq.copy()
    best_amount = 0
    best_path = []
    for (t1, t2), _ in l.items():
        if t1 == token and (t1, t2) not in visited:
            visited.add((t1, t2))
            next_amount = getAmountOut(amountIn, l[(t1, t2)][0], l[(t1, t2)][1])
            swap(t1, t2, amountIn, l)
            if next_amount > 0:
                amount, p = arbitrage(t2, next_amount, path + [t2], visited, l)
                if amount > best_amount:
                    best_amount = amount
                    best_path = p
            visited.remove((t1, t2))
    return best_amount, best_path




In [16]:
bp = arbitrage("tokenB", 5, ["tokenB"], set(), newLq)[1]


In [21]:
for i in range(len(bp) - 1):
    # address[] memory route = new address[](2);
    #     route[0] = address(tokenB);
    #     route[1] = address(tokenD);
    print(f"route[0] = address({bp[i]});")
    print(f"route[1] = address({bp[i+1]});")
    print(f"console.log({bp[i]}.balanceOf(arbitrager));")
    print(f"router.swapExactTokensForTokens({bp[i]}.balanceOf(arbitrager), 0, route, arbitrager, block.timestamp);")
    

route[0] = address(tokenB);
route[1] = address(tokenD);
console.log(tokenB.balanceOf(arbitrager));
router.swapExactTokensForTokens(tokenB.balanceOf(arbitrager), 0, route, arbitrager, block.timestamp);
route[0] = address(tokenD);
route[1] = address(tokenE);
console.log(tokenD.balanceOf(arbitrager));
router.swapExactTokensForTokens(tokenD.balanceOf(arbitrager), 0, route, arbitrager, block.timestamp);
route[0] = address(tokenE);
route[1] = address(tokenA);
console.log(tokenE.balanceOf(arbitrager));
router.swapExactTokensForTokens(tokenE.balanceOf(arbitrager), 0, route, arbitrager, block.timestamp);
route[0] = address(tokenA);
route[1] = address(tokenE);
console.log(tokenA.balanceOf(arbitrager));
router.swapExactTokensForTokens(tokenA.balanceOf(arbitrager), 0, route, arbitrager, block.timestamp);
route[0] = address(tokenE);
route[1] = address(tokenD);
console.log(tokenE.balanceOf(arbitrager));
router.swapExactTokensForTokens(tokenE.balanceOf(arbitrager), 0, route, arbitrager, block.timesta